In [7]:
from transformers import AutoTokenizer

model_path="/root/projects/ascend_r1_turtorial/output/qwen-3b-r1-coundown"

tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_r1_prompt(numbers, target):
    """
    生成 R1 Countdown 游戏提示词

    参数:
        numbers (list[int]): 数字列表
        target (int): 目标值
    返回:
        dict: 生成的一个数据样本
    """
    # 定义提示词前缀
    r1_prefix = [
        {
            "role": "user",
            "content": f"使用给定的数字 {numbers}，创建一个等于 {target} 的方程。你可以使用基本算术运算（+、-、*、/）一次或多次，但每个数字只能使用一次。在 <think> </think> 标签中展示你的思考过程，并在 <answer> </answer> 标签中返回最终方程，例如 <answer> (1 + 2) / 3 </answer>。在 <think> 标签中逐步思考。",
        },
        {
            "role": "assistant",
            "content": "让我们逐步解决这个问题。\n<think>",  # 结尾使用 `<think>` 促使模型开始思考
        },
    ]

    return  tokenizer.apply_chat_template(
            r1_prefix, tokenize=False, continue_final_message=True)

text=generate_r1_prompt([10,3,6],7)
print(text)

import torch

from transformers import AutoModelForCausalLM
device = "cuda:3"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map=device
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
使用给定的数字 [10, 3, 6]，创建一个等于 7 的方程。你可以使用基本算术运算（+、-、*、/）一次或多次，但每个数字只能使用一次。在 <think> </think> 标签中展示你的思考过程，并在 <answer> </answer> 标签中返回最终方程，例如 <answer> (1 + 2) / 3 </answer>。在 <think> 标签中逐步思考。<|im_end|>
<|im_start|>assistant
让我们逐步解决这个问题。
<think>


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 首先，我们需要找到一个组合，使得使用给定的数字 10, 3, 6 以及基本算术运算（+、-、*、/）可以得到结果 7。我们可以尝试不同的组合。假设我们尝试 10 - 3 + 6，计算得 10 - 3 = 7，然后 7 + 6 = 13，这不对。接下来尝试 10 - (3 + 6)，计算得 10 - 9 = 1，这也不对。再试 10 - 3 * 6，计算得 10 - 18 = -8，也不对。最后试 10 - 3 / 6，计算得 10 - 0.5 = 9.5，这也不对。继续尝试 10 + 3 - 6，计算得 10 + 3 = 13，然后 13 - 6 = 7，这正是我们要的结果。所以答案是 10 + 3 - 6。</think>
<answer> (10 + 3) - 6 </answer>
